In [1]:
import openpyxl
import glob
from pathlib import Path

In [2]:
import cv2 as cv
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import seaborn as sns
from scipy import stats
import os
import re
import pickle
import imutils
from scipy.signal import find_peaks, savgol_filter
from pathlib import Path
import glob
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from utils import noise_detection,  cfar_fast
from utils import find_periods

In [3]:
def init_condition(avis, x, y):
    capture = cv.VideoCapture(avis)
    _, frame1 = capture.read()
    
    # Crop to the selected 50x50 region
    frame1 = frame1[y:y+50, x:x+80]
    
    num_frames = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255  # Set saturation channel
    
    return capture, num_frames, hsv

In [4]:
def select_roi_interactive(avis):
    roi_x = 80
    roi_y = 50
    # Initialize video capture
    capture = cv.VideoCapture(avis)
    ret, frame = capture.read()
    if not ret:
        raise ValueError("Failed to read video")
    
    # Variables to store ROI coordinates
    roi_selected = False
    x_center, y_center = -1, -1

    # Mouse callback function
    def mouse_callback(event, x, y, flags, param):
        nonlocal roi_selected, x_center, y_center
        if event == cv.EVENT_LBUTTONDOWN:
            x_center, y_center = x, y
            roi_selected = True

    # Create window and set mouse callback
    cv.namedWindow("Select ROI Center")
    cv.setMouseCallback("Select ROI Center", mouse_callback)

    while True:
        display_frame = frame.copy()
        if roi_selected:
            # Ensure ROI stays within frame boundaries
            x = max(0, min(x_center - int(roi_x/2), frame.shape[1] - roi_x))
            y = max(0, min(y_center - int(roi_y/2), frame.shape[0] - roi_y))
            
            # Draw the 50x50 ROI rectangle
            cv.rectangle(display_frame, (x, y), (x+roi_x, y+roi_y), (0, 255, 0), 2)
            
            # Show cropped preview in a new window
            cropped_preview = frame[y:y+roi_y, x:x+roi_x]
            cv.imshow("Cropped Preview", cropped_preview)
        
        cv.imshow("Select ROI Center", display_frame)
        key = cv.waitKey(1) & 0xFF
        if key == ord('q') and roi_selected:  # Press 'q' to confirm
            break
        elif key == 27:  # Press ESC to exit
            cv.destroyAllWindows()
            capture.release()
            return None, None

    cv.destroyAllWindows()
    capture.release()
    
    # Final ROI coordinates (top-left corner)
    x = max(0, min(x_center - int(roi_x/2), frame.shape[1] - roi_x))
    y = max(0, min(y_center - int(roi_y/2), frame.shape[0] - roi_y))
    return x, y

In [5]:
trials = []
# Define variable to load the dataframe
dataframe = openpyxl.load_workbook("F:\\swim_motion_data\\selected_sets.xlsx")

# Define variable to read sheet
dataframe1 = dataframe.active

# Iterate the loop to read the cell values
for row in range(0, dataframe1.max_row):
    for col in dataframe1.iter_cols(1, 1):
        trials.append(col[row].value)

In [6]:
len(trials)

44

In [7]:
for trial in trials:
    date = trial.split("_")[1]
    Fx = trial.split("_")[2]
    
    avi = glob.glob(f"F:\\Temp_opto\\{date}\\**\\SIDECAMERA\\{Fx}\\**\\{trial}.avi")[0]
    
    save_dir = f"F:\\swim_motion_data\\lcr_kai\\{trial[5:-5]}\\SIDECAM"
    Path(save_dir).mkdir(parents = True, exist_ok = True)
    
    if Path(f"{save_dir}/cord.txt").exists():
       with open(f"{save_dir}/cord.txt", "r") as f:
           x_cord = int(f.readline().split()[-1])
           y_cord = int(f.readline().split()[-1])
    else:
        import winsound
        frequency = 2500  # Set Frequency To 2500 Hertz
        duration = 500  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
        x_cord, y_cord = select_roi_interactive(avi)
        with open(f"{save_dir}/cord.txt", "w") as f:
            f.write("x\t" + str(x_cord))
            f.write("\ny\t" + str(y_cord))

        
    capture, num_frames, hsv = init_condition(avi, x_cord, y_cord)
    res = list(range(0, num_frames))

    if Path(f"{save_dir}/x_shift.npy").exists():
        horizontal_motion = np.load(f"{save_dir}/x_shift.npy")
    else:
        horizontal_motion = []
    
        capture.set(cv.CAP_PROP_POS_FRAMES, 0)
        ret, prev_frame = capture.read()
        prev_frame = prev_frame[y_cord:y_cord+50, x_cord:x_cord+80]
        prev_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
        
        for frame in tqdm(range(1, num_frames), desc=f"Optic Flow for {trial[5:-5]}"):
            ret, curr_frame = capture.read()
            if not ret:
                break
                
            curr_frame = curr_frame[y_cord:y_cord+50, x_cord:x_cord+80]
            curr_gray = cv.cvtColor(curr_frame, cv.COLOR_BGR2GRAY)
            
            # Calculate dense optical flow
            flow = cv.calcOpticalFlowFarneback(
                prev_gray, curr_gray, 
                None, 0.5, 3, 15, 3, 5, 1.2, 0
            )
            
            # Focus on horizontal motion (x-axis component)
            horizontal_flow = flow[..., 0]
            avg_horizontal = np.mean(horizontal_flow)
            horizontal_motion.append(avg_horizontal)
            
            prev_gray = curr_gray
        horizontal_motion = -np.array(horizontal_motion) ###### make positive to upward motion 
        np.save(f"{save_dir}/x_shift.npy", horizontal_motion)